<a href="https://colab.research.google.com/github/manhhad32/nlp/blob/master/SearchUsePhoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers torch sklearn


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [20]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Tải PhoBERT-large và tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large")
model = AutoModel.from_pretrained("vinai/phobert-large")

# Sử dụng GPU nếu có
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 1024, padding_idx=1)
    (position_embeddings): Embedding(258, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      

In [21]:
def encode_text(text):
    tokens = tokenizer(text, return_tensors='pt', truncation=True, max_length=100).to(device)
    with torch.no_grad():
        outputs = model(**tokens)
    # Tính trung bình các vector đầu ra cho tất cả các tokens
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

# Hàm để mã hóa văn bản dài bằng cách chia thành các chunk nhỏ
def encode_long_text(text, chunk_size=512):
    tokens = tokenizer.encode(text, truncation=False)
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]

    chunk_vectors = []
    for chunk in chunks:
        chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
        chunk_vector = encode_text(chunk_text)
        chunk_vectors.append(chunk_vector)

    # Tính trung bình của tất cả các vector chunk để biểu diễn toàn bộ văn bản
    return torch.tensor(chunk_vectors).mean(dim=0).numpy()

# Hàm tính độ tương đồng cosine
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


In [25]:
import glob
import os
import re

# Đường dẫn đến thư mục chứa các file document
doc_folder = '/content/drive/MyDrive/datanlp/'

# Đọc và mã hóa tất cả các tài liệu trong thư mục
document_vectors = []
documents = []
fi_names = []
def load_documents_from_directory(directory):
    print('Loading documents from directory')
    docs = []
    for file_name in os.listdir(directory):
        fi_names.append(file_name)
        print('Đọc dữ liệu từ file: [{}]...'.format(file_name))
        file_path = os.path.join(directory, file_name)
        lines = []
        with open(file_path, 'r', encoding='utf-8') as fi:
            # reade file acording lines.
            for line in fi:
                # change to lower character and remove space at head/tail of line.
                line = line.lower().strip()
                # add to list
                lines.append(line)
        # create doc as text inline.
        doc = " ".join(lines)
        # clean, remove some spec word in head/tail.
        re.sub('\W+',' ', doc)
        docs.append(doc)
    print('End Loading documents from directory')
    return docs

documents = load_documents_from_directory(doc_folder)

for doc in documents:
  document_vectors.append(encode_long_text(doc))
#for filepath in glob.glob(doc_folder):
#    with open(filepath, 'r', encoding='utf-8') as file:
#        text = file.read()
#        documents.append(text)
#        document_vectors.append(encode_long_text(text))

document_vectors = torch.tensor(document_vectors)



Loading documents from vnexpress directory
Đọc dữ liệu từ file: [ba-ly-do-nen-di-phu-quoc-ngay-va-luon-4528757.txt]...
Đọc dữ liệu từ file: [banh-lot-xao-trung-mon-an-campuchia-hut-khach-viet-4527571.txt]...
Đọc dữ liệu từ file: [ba-mon-ca-cho-bua-toi-se-lanh-o-dien-bien-4527828.txt]...
Đọc dữ liệu từ file: [6-khach-san-viet-vao-top-sang-trong-nhat-dong-nam-a-4529357.txt]...
Đọc dữ liệu từ file: [10-khu-luu-tru-viet-thang-giai-sang-trong-cua-chau-a-4527584.txt]...
Đọc dữ liệu từ file: [ben-trong-du-thuyen-dau-gia-hon-35-ty-dong-cua-flc-4527411.txt]...
Đọc dữ liệu từ file: [cac-diem-den-bi-an-o-viet-nam-cho-ngay-halloween-4527952.txt]...
Đọc dữ liệu từ file: [chang-ha-noi-tp-hcm-co-quay-check-in-va-nhan-hanh-ly-rieng-4529038.txt]...
Đọc dữ liệu từ file: [cach-sun-group-nang-tam-du-lich-hon-thom-4528015.txt]...
Đọc dữ liệu từ file: [cho-ngoi-tot-va-te-nhat-tren-may-bay-4528373.txt]...
Đọc dữ liệu từ file: [bon-yeu-cau-khien-tiep-vien-kho-chiu-nhat-4528270.txt]...
Đọc dữ liệu từ file: [co

In [34]:
# Mã hóa câu truy vấn
query = "Hà Nội, TP HCM, Phú Yên, Quảng Ninh và Phú Quốc là những nơi có các khu nghỉ dưỡng, khách sạn được vinh danh tại lễ trao giải World Luxury Hotel Awards."
query_vector = encode_long_text(query)

# Tính toán Cosine Similarity
similarities = []
for doc_vector in document_vectors:
    similarities.append(cosine_similarity(query_vector, doc_vector))


# Sắp xếp các tài liệu theo độ tương đồng giảm dần
doc_fi_names = zip(documents, fi_names)
sorted_docs = sorted(zip(doc_fi_names, similarities), key=lambda x: x[1], reverse=True)

# In ra các tài liệu có độ tương đồng cao nhất
top_k = 5  # Số lượng tài liệu muốn hiển thị
result_doc = []
for doc, similarity in sorted_docs[:top_k]:
    result_doc.append(f"Tài liệu: {tuple(doc)[1]}... (độ tương đồng: {similarity:.4f})")
    print(f"Tài liệu: {tuple(doc)[1]}... (độ tương đồng: {similarity:.4f})")


Tài liệu: 6-khach-san-viet-vao-top-sang-trong-nhat-dong-nam-a-4529357.txt... (độ tương đồng: 0.9905)
Tài liệu: 10-khu-luu-tru-viet-thang-giai-sang-trong-cua-chau-a-4527584.txt... (độ tương đồng: 0.9886)
Tài liệu: cach-sun-group-nang-tam-du-lich-hon-thom-4528015.txt... (độ tương đồng: 0.9868)
Tài liệu: mat-trai-cua-trao-luu-cam-trai-4529913.txt... (độ tương đồng: 0.9866)
Tài liệu: huong-di-cho-quang-nam-phat-trien-du-lich-trai-mua-4516307.txt... (độ tương đồng: 0.9862)
